In [1]:
# Input
depth = 7740
target = (12, 763)

In [2]:
from functools import lru_cache

def geological_index(pos):
    if pos == target:
        return 0
    x, y = pos
    if x < 0 or y < 0:
        return 0
    if y == 0:
        return (x * 16807) % 20183
    if x == 0:
        return (y * 48271) % 20183
    return (erosion_level((x-1, y)) * erosion_level((x, y-1))) % 20183

@lru_cache(maxsize=None)
def erosion_level(pos):
    return (geological_index(pos) + depth) % 20183

def region_type(pos):
    return erosion_level(pos) % 3

In [3]:
risk = 0
for y in range(target[1] + 1):
    for x in range(target[0] + 1):
        risk += region_type((x, y)) % 3

print("Part 1: {}".format(risk))

Part 1: 9899


In [4]:
import heapq

def heuristic(pos):
    return abs(target[0] - pos[0]) + abs(target[1] - pos[1])

def neighbors(pos):
    x, y = pos
    result = [(x+1, y), (x, y+1)]
    if x > 0:
        result.append((x-1, y))
    if y > 0:
        result.append((x, y-1))
    return result

def move(pos, neighbor, tool):
    c1 = region_type(pos)
    c2 = region_type(neighbor)
    cost = 1
    if tool == c1 or tool == c2:
        # We need to change the tool
        tool = [x for x in range(3) if x != c1 and x != c2][0]
        cost += 7
    return cost, tool

def find_path():
    done = set()
    queue = []
    start = (0, 0)
    # {(Position, current tool): (distance from start, previous node)}
    paths = {(start, 1): (0, None)}
    heapq.heappush(queue, (heuristic(start), start, 1))
    
    results = []
    
    while queue:
        h, pos, tool = heapq.heappop(queue)
        cost, previous = paths[(pos, tool)]
        done.add((pos, tool))
        if pos == target:
            return cost

        for neighbor in neighbors(pos):
            if neighbor in done:
                continue
            move_cost, new_tool = move(pos, neighbor, tool)
            new_cost = cost + move_cost
            if neighbor == target and tool != 1:
                new_cost += 7
            if (neighbor, new_tool) in paths:
                old_cost, old_previous = paths[(neighbor, new_tool)]
                if old_cost <= new_cost:
                    continue
            
            paths[(neighbor, new_tool)] = (new_cost, pos)
            heapq.heappush(queue, (new_cost + heuristic(neighbor), neighbor, new_tool))

In [5]:
print("Part 2: {}".format(find_path()))

Part 2: 1051
